In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
print(os.getcwd())
os.chdir('/content/drive/MyDrive/')

/content


###Reminder to remove LOG from modules.py and functions.py. Later on might have to remove it from every file (or reinstate log)

In [ ]:
import numpy as np

class PropagationPath:
    def __init__(self):
        self.points = []
        self.interaction_types = []   # 0 - initial transmitter (radio antenna)   1 - final point (user)   2 - specular reflection   3 - diffraction around the edge
        self.path_gain_db = 0   # dB = 10 * log10(I/I0)   -20 dB = 100 times weaker   -90 dB = 1000000000 times weaker
        self.hash = 0   # hashed set of interactions (objects / surfaces)

offset = 0
def read_int():
    global offset, bytes
    offset += 4
    return bytes[offset-4:offset].copy().view(np.int32)[0]
def read_uint():
    global offset, bytes
    offset += 4
    return bytes[offset-4:offset].copy().view(np.uint32)[0]
def read_float():
    global offset, bytes
    offset += 4
    return bytes[offset-4:offset].copy().view(np.float32)[0]

def read_file(filename: str):
    global offset, bytes
    offset = 0
    bytes = np.fromfile(filename, dtype=np.uint8)

    transmitter_count = read_int()
    paths = []
    for tx in range(transmitter_count):
        receiver_count = read_int()
        paths.append([])
        for rx in range(receiver_count):
            path_count = read_int()
            paths[tx].append([])
            for p in range(path_count):
                path = PropagationPath()
                point_count = read_int()
                for _ in range(point_count):
                    x = read_float()
                    y = read_float()
                    z = read_float()
                    path.points.append((x, y, z))
                for _ in range(point_count):
                    interaction = read_int()
                    path.interaction_types.append(interaction)

                path.path_gain_db = read_float()
                path.hash = read_uint()
                paths[tx][rx].append(path)
    return paths

if __name__ == "__main__":
    path_file = 'eu_city_2x2_macro_306.bin'
    paths = read_file(path_file)


In [ ]:
print(paths[0])
print(len(paths))
print(len(paths[0]))
print(len(paths[1]))

[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [],

In [ ]:
#Find first non-zero path
data_list=[]
for i in range(len(paths[0])):
  if len(paths[0][i])!=0:
    for k in paths[0][i]:
      print(k)
    break
  #break

## Testing
### Only using 1 data sample right now. Trying to get it to work before looking at bigger batch sizes

In [ ]:
print(paths[0][7182][1].points)
print(paths[0][7182][1].interaction_types)
print(paths[0][7182][1].path_gain_db)
print(paths[0][7182][1].hash)

[(158.49117, -32.520683, 53.320007), (-342.37683, -341.62332, 56.24), (-348.43402, -345.3433, 56.24), (-360.13348, -351.58405, 43.58853), (-371.9333, -369.0092, 23.50962)]
[0, 3, 3, 2, 1]
-148.83165
837731744


In [ ]:
!pip install ttools

  Preparing metadata (setup.py) ... done
  Created wheel for ttools: filename=ttools-0.1-py3-none-any.whl size=1205 sha256=7a428109f50f994e40432f19e95aa51961bb1aecc56fe0feae97821a155a34f3
  Stored in directory: /root/.cache/pip/wheels/e2/ed/d7/89a4523eca26412a0092b3d5395284fd0f3e0cceaedf8176cf
Successfully built ttools


In [ ]:
import torch
import torch.nn as nn
#import sys
#sys.path.insert(1, '/content/drive/MyDrive/sbmcFiles/')
from sbmcFiles import modules as ops

In [ ]:
a=paths[0][7182][1]#4dim

In [ ]:
data_arrays = np.array(a.points)  # Convert the list of tuples to a NumPy array
data_tensor = torch.tensor(data_arrays)  # Convert the NumPy array to a PyTorch tensor
data_indices = torch.tensor(a.interaction_types, dtype=torch.long)  # Convert the list of integers to a PyTorch tensor
float_gain = torch.tensor(a.path_gain_db, dtype=torch.float32)  # Convert the float to a PyTorch tensor
float_hash= torch.tensor(a.hash, dtype=torch.long)

In [ ]:

embedding=ops.ConvChain(len(paths[0][7182][1].points),64,width=64,depth=3,ksize=1, pad=False)

embedding=ops.ConvChain(1,64,width=64,depth=3,ksize=1, pad=False)

In [ ]:
print(len(a.points))
print(embedding)

5
ConvChain(
  (layer_0): _ConvBNRelu(
    (layer): Sequential(
      (0): Conv2d(1, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): ReLU()
    )
  )
  (layer_1): _ConvBNRelu(
    (layer): Sequential(
      (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): ReLU()
    )
  )
  (prediction): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
)


In [ ]:
propagation = ops.Autoencoder(64, 64, num_levels=3,
                increase_factor=2.0, num_convs=3, width=64, ksize=3,
                output_type="leaky_relu", pooling="max")

In [ ]:
def init_data(paths, batch_size, ratio):

    all_paths = []

    for i in range(len(paths)):
      for j in range(len(paths[0])):
        if len(paths[i][j]) != 0:
          for k in paths[i][j]:
            #all_paths.append(k)
            data_tensor = torch.tensor(k.points, dtype=torch.float32)
            labels_tensor = torch.tensor(k.interaction_types, dtype=torch.float32)
            gain_tensor = torch.tensor(k.path_gain_db, dtype=torch.float32)
            hash_tensor = torch.tensor(k.hash, dtype=torch.int64)

            #print(labels_tensor.shape,gain_tensor.shape)
            #print(gain_tensor)
            #print(labels_tensor)
            #print(data_tensor)

            #combined_tensor = torch.cat((data_tensor, labels_tensor.unsqueeze(1), gain_tensor.unsqueeze(0)), dim=1)
            combined_tensor = torch.cat((data_tensor, labels_tensor.unsqueeze(1)), dim=1)
            gain = gain_tensor.view(1, 1)
            gain = gain.expand_as(combined_tensor[:, :1])
            final_combined_tensor = torch.cat((combined_tensor, gain), dim=1)
            all_paths.append(final_combined_tensor)

    #rnd_int = np.random.randint(0, len(all_paths))

    datasets = {}
    datasets['train'] = all_paths[:int(len(all_paths) * ratio)]
    datasets['val'] = all_paths[int(len(all_paths) * ratio):]

    # Shuffle data
    dataset_length = len(datasets['train'])
    indices = list(range(dataset_length))
    np.random.shuffle(indices)
    sampler = torch.utils.data.SubsetRandomSampler(indices)

    # Create DataLoaders using these samplers
    dataloaders = {
        'train': torch.utils.data.DataLoader(
            datasets['train'],
            batch_size=batch_size,
            sampler=sampler,
            num_workers=2,
            pin_memory=False,
        ),
        'val': torch.utils.data.DataLoader(
            datasets['val'],
            batch_size=batch_size,
            shuffle=True,
            num_workers=2,
            pin_memory=False,
        )
    }

    return datasets, dataloaders

In [ ]:
dataset, dataloaders = init_data(paths, 10, 0.9)

In [ ]:
print("train data length: "+str(len(dataset['train'])))
print("val data length: "+str(len(dataset['val'])))

train data length: 681758
val data length: 75751


In [ ]:
for inputs in dataloaders['train']:
  inputs = inputs.to(device)
  print(inputs)
  interface.train_batch(inputs, grad_hook_mode=False)

## Class Approach to embedding and rest of forward function

In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset

# Define a custom dataset
class MyDataset(Dataset):
    def __init__(self, paths, ratio, batch_size,mode='train'):
        self.paths = paths
        self.mode = mode
        self.ratio = ratio
        self.batch_size = batch_size
        self.all_paths = []

        for i in range(len(self.paths)):
            for j in range(len(self.paths[0])):
                if len(self.paths[i][j]) != 0:
                    for k in self.paths[i][j]:
                        #self.all_paths.append(k)

                        data_tensor = torch.tensor(k.points, dtype=torch.float32)
                        labels_tensor = torch.tensor(k.interaction_types, dtype=torch.float32)
                        gain_tensor = torch.tensor(k.path_gain_db, dtype=torch.float32)
                        hash_tensor = torch.tensor(k.hash, dtype=torch.int64)

                        #print(labels_tensor.shape,gain_tensor.shape)
                        #print(gain_tensor)
                        #print(labels_tensor)
                        #print(data_tensor)

                        #combined_tensor = torch.cat((data_tensor, labels_tensor.unsqueeze(1), gain_tensor.unsqueeze(0)), dim=1)
                        combined_tensor = torch.cat((data_tensor, labels_tensor.unsqueeze(1)), dim=1)
                        gain = gain_tensor.view(1, 1)
                        gain = gain.expand_as(combined_tensor[:, :1])
                        final_combined_tensor = torch.cat((combined_tensor, gain), dim=1)

                        if final_combined_tensor.shape[0] < 5:
                            # Calculate the number of rows to pad
                            num_rows_to_pad = 5 - final_combined_tensor.shape[0]

                            # Pad the tensor along the top with zeros
                            padded_tensor = F.pad(final_combined_tensor, (0, 0,0, num_rows_to_pad), value=0)
                        else:
                            # If the tensor already has the target number of rows, no padding is needed
                            padded_tensor = final_combined_tensor


                        self.all_paths.append(padded_tensor)

        dataset_length = len(self.all_paths)
        split_idx = int(dataset_length * self.ratio)
        self.datasets = {
            'train': self.all_paths[:split_idx],
            'validation': self.all_paths[split_idx:]
        }

    def __len__(self):
      if self.mode == 'train':
          return len(self.datasets['train'])
      elif self.mode == 'validation':
          return len(self.datasets['validation'])
      else:
          raise ValueError("Invalid mode. Use 'train' or 'validation'.")
      #return len(self.datasets['train'])

    def __getitem__(self, idx):
        """data_point = self.datasets['train'][idx]
        data_tensor = torch.tensor(data_point.points, dtype=torch.float32)
        labels_tensor = torch.tensor(data_point.interaction_types, dtype=torch.float32)
        gain_tensor = torch.tensor(data_point.path_gain_db, dtype=torch.float32)
        hash_tensor = torch.tensor(data_point.hash, dtype=torch.int64)

        #print(labels_tensor.shape,gain_tensor.shape)
        #print(gain_tensor)
        #print(labels_tensor)
        #print(data_tensor)

        #combined_tensor = torch.cat((data_tensor, labels_tensor.unsqueeze(1), gain_tensor.unsqueeze(0)), dim=1)
        combined_tensor = torch.cat((data_tensor, labels_tensor.unsqueeze(1)), dim=1)
        gain = gain_tensor.view(1, 1)
        gain = gain.expand_as(combined_tensor[:, :1])
        final_combined_tensor = torch.cat((combined_tensor, gain), dim=1)
        return final_combined_tensor
        """

        if self.mode == 'train':
            data_item = self.datasets['train'][idx]
        elif self.mode == 'validation':
            data_item = self.datasets['validation'][idx]
        else:
            raise ValueError("Invalid mode. Use 'train' or 'validation'.")
        return data_item

    def test(self):
      return self.datasets['train'][0]


In [ ]:
dataset= MyDataset(paths,0.8,12)

In [ ]:
test=dataset.__getitem__(19)
dataset.__getitem__(19)

tensor([[ 158.4912,  -32.5207,   53.3200,    0.0000, -132.9013],
        [ -43.7971,  -72.5437,   53.0900,    3.0000, -132.9013],
        [-109.2309,  -98.4227,   32.5367,    2.0000, -132.9013],
        [ -99.5067,  -90.6448,   28.8996,    1.0000, -132.9013],
        [   0.0000,    0.0000,    0.0000,    0.0000,    0.0000]])

In [ ]:
train_loader = DataLoader(dataset, batch_size=4, shuffle=False)

In [ ]:
embedding=ops.ConvChain(1,64,width=64,depth=3,ksize=1, pad=False)
propagation = ops.Autoencoder(64, 64, num_levels=3,
                increase_factor=2.0, num_convs=3, width=64, ksize=3,
                output_type="leaky_relu", pooling="max")
final = ops.ConvChain(128, 1, width=128,
                depth=2, ksize=1, pad=False, output_type="leaky_relu")

In [ ]:
#print(train_loader)
for batch in train_loader:
  #print(batch)
  #print(batch.unsqueeze(1))
  flat=embedding(batch.unsqueeze(1))
  print("Input shape: "+str(batch.unsqueeze(1).shape))
  print("Output shape from Embedding: "+str(flat.shape))

  propagated=propagation(flat)
  print("Output shape from Propagation: "+str(propagated.shape))

  test = torch.cat([flat, propagated], dim=1)
  outp=final(test)
  print("Output shape after Final: "+str(outp.shape))
  print(outp)
  break


Input shape: torch.Size([4, 1, 5, 5])
Output shape from Embedding: torch.Size([4, 64, 5, 5])
Output shape from Propagation: torch.Size([4, 64, 5, 5])
Output shape after Final: torch.Size([4, 1, 5, 5])
tensor([[[[-2.9129e-01,  5.8539e+00, -9.8553e-02, -7.4080e-04,  2.6850e+01],
          [ 6.0884e+01,  6.3342e+01, -1.0403e-01, -6.0884e-03,  2.6864e+01],
          [ 6.2645e+01,  6.4487e+01, -1.0381e-01, -6.4704e-03,  2.6843e+01],
          [ 6.8097e+01,  6.7577e+01, -4.4120e-02, -2.7630e-03,  2.6879e+01],
          [-1.8939e-04, -9.8763e-05, -1.0406e-03, -6.3835e-04, -1.7150e-04]]],


        [[[-2.9124e-01,  5.8469e+00, -9.8478e-02, -7.5888e-04,  2.7241e+01],
          [ 6.2631e+01,  6.2500e+01, -1.0409e-01, -6.0872e-03,  2.7231e+01],
          [ 6.3753e+01,  6.3194e+01, -1.0392e-01, -6.6701e-03,  2.7208e+01],
          [ 6.5941e+01,  6.4375e+01, -8.0795e-02, -4.7023e-03,  2.7257e+01],
          [ 6.8108e+01,  6.7634e+01, -4.4266e-02, -2.4886e-03,  2.7252e+01]]],


        [[[-2.9124e-0


####This was in source code:

flat = torch.cat([flat.view([bs * spp, self.intermc, h, w]), propagated.unsqueeze(1).repeat(
            [1, spp, 1, 1, 1]).view(bs*spp, self.intermc, h, w)], 1)
        
        
out = self.final(flat).view([bs, spp, self.outc, h, w])

###
the code concatenates output from embedding and output from propagation. they add spp dimension to propagated to then find batch size by squeezing bs * spp. the concat is sent to final and then stretched out to spp dimension again, I think. The reason for squeezing to 4 dim in source code is because it is needed for the conv2d functions. Since we already had 4 dim I did not squeeze the dims inbetween the functions.


In [ ]:
propagation = ops.Autoencoder(64, 64, num_levels=3,
                increase_factor=2.0, num_convs=3, width=64, ksize=3,
                output_type="leaky_relu", pooling="max")

In [ ]:
propagated=propagation(flat)

In [ ]:
print(propagated.shape)
#print(test)

torch.Size([4, 64, 5, 5])


In [ ]:
final = ops.ConvChain(128, 1, width=128,
                depth=2, ksize=1, pad=False, output_type="leaky_relu")

In [ ]:
test = torch.cat([flat, propagated], dim=1)

In [ ]:
outp=final(test)

In [ ]:
print(outp.shape)

torch.Size([4, 1, 5, 5])


In [ ]:
print(outp)

tensor([[[[ 4.0564e-02, -2.1261e-03,  1.4312e-02, -3.9845e-05, -9.6248e-03],
          [-2.1805e-02, -2.2671e-02,  9.5891e-03, -6.5805e-05, -9.6346e-03],
          [-2.2484e-02, -2.3116e-02,  2.1243e-03, -8.6751e-05, -9.6410e-03],
          [-2.4369e-02, -2.4183e-02, -3.5524e-05, -8.8116e-05, -9.6372e-03],
          [-3.4554e-05, -7.9216e-05, -7.6091e-05, -7.4608e-05, -4.0877e-05]]],


        [[[ 4.0346e-02, -2.1231e-03,  1.5456e-02, -4.0859e-05, -9.7705e-03],
          [-2.2406e-02, -2.2331e-02,  1.0435e-02, -6.3362e-05, -9.7501e-03],
          [-2.2882e-02, -2.2687e-02,  5.2480e-03, -5.4038e-05, -9.7807e-03],
          [-2.3619e-02, -2.3015e-02, -7.3094e-06, -8.2924e-05, -9.7629e-03],
          [-2.4333e-02, -2.4193e-02, -3.4548e-06, -5.4310e-05, -9.7630e-03]]],


        [[[ 4.0347e-02, -2.1231e-03,  1.5456e-02, -4.0858e-05, -9.7706e-03],
          [-2.2405e-02, -2.2332e-02,  1.0435e-02, -6.3359e-05, -9.7501e-03],
          [-2.2882e-02, -2.2688e-02,  5.2486e-03, -5.4040e-05, -9.78